In [1]:
import sys
import os
import tensorflow as tf 
import numpy as np
import time

In [2]:
filename = 'C:\\Users\\rober\\Practicas\\training\\training_v2.tfrecords'

In [3]:
def read_and_decode(filename_queue):
    #Definimos la ruta de los tfrecords
    filename = 'C:\\Users\\rober\\Practicas\\training\\training_v2.tfrecords'
    feature={
        'image/encoded': tf.FixedLenFeature([], tf.string,default_value=''),
        'label/encoded': tf.FixedLenFeature([], tf.string,default_value='')
    }
    #Función encargada de añadir a una cola FIFO la lista de ficheros, además el parametro epochs
    #nos proporciona el numero de tensores que obtendremos en cada iteracción
    filename_queue = tf.train.string_input_producer([filename], num_epochs=50)
    #Definimos un lector
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example, features=feature)
    #Diccionario con las características necesarias.Decodificamos el "record" leido
    #por el lector
    #features = tf.parse_single_example(serialized_example,)  
    # Nos devuelve la imagen y la etiqueta
    #Convierte la etiqueta a su formato original 
    with tf.name_scope('label'):
        label = tf.decode_raw(features['label/encoded'], tf.float32)
        label = tf.transpose(tf.reshape(label, [1,240,240]), (0,2,1))
    #Convierte la imagen a su formato original 
    with tf.name_scope('image'):
        image = tf.decode_raw(features['image/encoded'], tf.float32)
        image = tf.transpose(tf.reshape(image, [4,240,240]), (0,2,1))
        #image = tf.reshape(image, [32, 100, 3])
    #Realiza el casteo a float32 e int32
    image = tf.cast(image, tf.float32)
    label = tf.cast(label, tf.int32)
    print("image", image.get_shape(), label.get_shape())
    
    return image,label

In [4]:
def inputs(batch_size,num_epochs,filename):
    if not num_epochs: num_epochs = None
    with tf.name_scope('input'):
        filename_queue = tf.train.string_input_producer([filename], num_epochs=num_epochs)
        image, label = read_and_decode(filename_queue=filename_queue)
        images, sparse_labels = tf.train.shuffle_batch(
        [image, label], batch_size=batch_size, num_threads=2,
        capacity=15,
        min_after_dequeue=10)
        return images, sparse_labels

In [5]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)
def maxpool2d(x, kernel_size=3,strides=1):
    return tf.nn.max_pool(x, ksize=[1, kernel_size, kernel_size, 1], 
                          strides=[1, strides, strides, 1],padding='SAME')
def deconv2d(x,W,output_shape,strides=1):
    return tf.nn.conv2d_transpose(x,W,output_shape,strides=[1,strides,strides,1],padding='SAME')
def print_tensor_shape(tensor,string):
    print(string,tensor.get_shape())

In [6]:
def loss(logits,labels):
        labels = tf.to_int64(labels)
        print_tensor_shape(logits,"Logits antes del reshape: ")
        print_tensor_shape(labels,"Labels antes del reshape: ")
        #OJO CON EL NUMERO DE CLASES [-1,NUMERO CLASES]
        logits_re = tf.reshape(logits,[-1,5])
        labels_re = tf.reshape(labels,[-1])
        print_tensor_shape(logits_re,"Logits despues del reshape: ")
        print_tensor_shape(labels_re,"Labels despues del reshape: ")
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_re,logits=logits_re,
                                                                       name='cross_entropy')
        loss = tf.reduce_mean(cross_entropy,name='cross_entropy_mean')
        return loss

In [7]:
def training(loss,learning_rate,decay_steps,decay_rate):
    tf.summary.scalar(loss.op.name,loss)
    global_step = tf.Variable(0,name='global_step',trainable=False)
    lr = tf.train.exponential_decay(learning_rate,
                                   global_step,
                                   decay_steps,
                                   decay_rate,
                                   staircase=True)
    tf.summary.scalar('learning_rate',lr)
    optimizer = tf.train.GradientDescentOptimizer(lr)
    train_op=optimizer.minimize(loss,global_step=global_step)
    return train_op

In [8]:
# Store layers weight & bias
weights = {
    #PESOS DEL PATH SMALL
    # 3x3 conv, 4 input, 64 outputs
    'w_s1': tf.Variable(tf.random_normal([3, 3, 4, 64])),
    # 3x3 conv, 64 inputs, 64 outputs
    'w_s2': tf.Variable(tf.random_normal([3, 3, 64, 64])),
    # 3x3 conv, 64 inputs, 64 outputs
    'w_s3': tf.Variable(tf.random_normal([3, 3, 64, 64])),
    # 3x3 conv, 64 inputs, 64 outputs
    'w_s4': tf.Variable(tf.random_normal([3, 3, 64, 64])),
    # 3x3 conv, 64 inputs, 64 outputs
    'w_s5': tf.Variable(tf.random_normal([3, 3, 64, 64])),
    # 3x3 conv, 4 input, 64 outputs
    'w_s6': tf.Variable(tf.random_normal([3, 3, 4, 64])),
    # 3x3 conv, 64 inputs, 64 outputs
    'w_s7': tf.Variable(tf.random_normal([3, 3, 64, 64])),
    # 3x3 conv, 64 inputs, 64 outputs
    'w_s8': tf.Variable(tf.random_normal([3, 3, 64, 64])),
    # 3x3 conv, 64 inputs, 64 outputs
    'w_s9': tf.Variable(tf.random_normal([3, 3, 64, 64])),
    
    #PESOS DEL PATH MEDIUM
    # 7x7 conv, 4 input, 128 outputs
    'w_m1': tf.Variable(tf.random_normal([7, 7, 4, 128])),
    # 7x7 conv, 128 inputs, 128 outputs
    'w_m2': tf.Variable(tf.random_normal([7, 7, 128, 128])),
    # 7x7 conv, 128 inputs, 128 outputs
    'w_m3': tf.Variable(tf.random_normal([7, 7, 128, 128])),
    # 7x7 conv, 128 inputs, 128 outputs
    'w_m4': tf.Variable(tf.random_normal([7, 7, 128, 128])),
    # 7x7 conv, 128 inputs, 128 outputs
    'w_m5': tf.Variable(tf.random_normal([7, 7, 128, 128])),
    # 7x7 conv, 4 input, 128 outputs
    'w_m6': tf.Variable(tf.random_normal([7, 7, 4, 128])),
    # 7x7 conv, 128 inputs, 128 outputs
    'w_m7': tf.Variable(tf.random_normal([7, 7, 128, 128])),
    # 7x7 conv, 128 inputs, 128 outputs
    'w_m8': tf.Variable(tf.random_normal([7, 7, 128, 128])),
    # 7x7 conv, 128 inputs, 128 outputs
    'w_m9': tf.Variable(tf.random_normal([7, 7, 128, 128])),
    
    #PESOS DEL PATH LARGE
    # 11x11 conv, 4 input, 256 outputs
    'w_l1': tf.Variable(tf.random_normal([11,11, 4, 256])),
    # 11x11 conv, 256 inputs, 256 outputs
    'w_l2': tf.Variable(tf.random_normal([11, 11, 256, 256])),
    # 11x11 conv, 256 inputs, 256 outputs
    'w_l3': tf.Variable(tf.random_normal([11, 11, 256, 256])),
    # 11x11 conv, 256 inputs, 256 outputs
    'w_l4': tf.Variable(tf.random_normal([11, 11, 256, 256])),
    # 11x11 conv, 256 inputs, 256 outputs
    'w_l5': tf.Variable(tf.random_normal([11, 11, 256, 256])),
    # 11x11 conv, 4 input, 256 outputs
    'w_l6': tf.Variable(tf.random_normal([11,11, 4, 256])),
    # 11x11 conv, 256 inputs, 256 outputs
    'w_l7': tf.Variable(tf.random_normal([11, 11, 256, 256])),
    # 11x11 conv, 256 inputs, 256 outputs
    'w_l8': tf.Variable(tf.random_normal([11, 11, 256, 256])),
    # 11x11 conv, 256 inputs, 256 outputs
    'w_l9': tf.Variable(tf.random_normal([11, 11, 256, 256])),
    
    #PESOS DEL PATH FUSION
    # 1x1 conv, 448 inputs, 300 outputs
    'w_f1': tf.Variable(tf.random_normal([1, 1, 448, 300])),
    # 1x1 conv, 300 inputs, 512 outputs
    'w_f2': tf.Variable(tf.random_normal([1, 1, 300, 512])),
    # 1x1 conv, 512 inputs, 5 outputs
    'w_f3': tf.Variable(tf.random_normal([1, 1, 512, 5])),
    # 3x3 conv, 448 inputs, 300 outputs
    'w_f4': tf.Variable(tf.random_normal([3, 3, 448, 300])),
    # 1x1 conv, 300 inputs, 512 outputs
    'w_f5': tf.Variable(tf.random_normal([1, 1, 300, 512])),
    # 1x1 conv, 512 inputs, 5 outputs
    'w_f6': tf.Variable(tf.random_normal([1, 1, 512, 5])),
    
    #PESOS DEL PATH DECONVOLUCIONAL
    # 1x1 conv, 5 inputs, 5 outputs
    'w_fdec1': tf.Variable(tf.random_normal([1, 1, 5, 5])),
    # 1x1 conv, 5 inputs, 5 outputs
    'w_fdec2': tf.Variable(tf.random_normal([1, 1, 5, 5])),
    
    #PESOS DEL PATH FINAL FUSION
    # 1x1 conv, 10 inputs, 5 outputs
    'w_final_fusion': tf.Variable(tf.random_normal([1, 1, 10, 5])),
    
}


biases = {
    #BIASES DEL PATH SMALL
    'b_s_1': tf.Variable(tf.random_normal([64])),
    'b_s_2': tf.Variable(tf.random_normal([64])),
    'b_s_3': tf.Variable(tf.random_normal([64])),
    'b_s_4': tf.Variable(tf.random_normal([64])),
    'b_s_5': tf.Variable(tf.random_normal([64])),
    'b_s_6': tf.Variable(tf.random_normal([64])),
    'b_s_7': tf.Variable(tf.random_normal([64])),
    'b_s_8': tf.Variable(tf.random_normal([64])),
    'b_s_9': tf.Variable(tf.random_normal([64])),
    
    #BIASES DEL PATH MEDIUM
    'b_m_1': tf.Variable(tf.random_normal([128])),
    'b_m_2': tf.Variable(tf.random_normal([128])),
    'b_m_3': tf.Variable(tf.random_normal([128])),
    'b_m_4': tf.Variable(tf.random_normal([128])),
    'b_m_5': tf.Variable(tf.random_normal([128])),
    'b_m_6': tf.Variable(tf.random_normal([128])),
    'b_m_7': tf.Variable(tf.random_normal([128])),
    'b_m_8': tf.Variable(tf.random_normal([128])),
    'b_m_9': tf.Variable(tf.random_normal([128])),
    
    #BIASES DEL PATH LARGE
    'b_l_1': tf.Variable(tf.random_normal([256])),
    'b_l_2': tf.Variable(tf.random_normal([256])),
    'b_l_3': tf.Variable(tf.random_normal([256])),
    'b_l_4': tf.Variable(tf.random_normal([256])),
    'b_l_5': tf.Variable(tf.random_normal([256])),
    'b_l_6': tf.Variable(tf.random_normal([256])),
    'b_l_7': tf.Variable(tf.random_normal([256])),
    'b_l_8': tf.Variable(tf.random_normal([256])),
    'b_l_9': tf.Variable(tf.random_normal([256])),
    
    #BIASES DEL PATH FUSION
    'b_f_1': tf.Variable(tf.random_normal([300])),
    'b_f_2': tf.Variable(tf.random_normal([512])),
    'b_f_3': tf.Variable(tf.random_normal([5])),
    'b_f_4': tf.Variable(tf.random_normal([300])),
    'b_f_5': tf.Variable(tf.random_normal([512])),
    'b_f_6': tf.Variable(tf.random_normal([5])),
    
    #BIASES DEL PATH FINAL FUSION
    'b_ff': tf.Variable(tf.random_normal([5]))
}
dropout = 0.5

In [9]:
def net(x,weights,biases,dropout):
    print_tensor_shape(x,'X antes del Reshaping')
    #x = tf.reshape(x, shape=[-1, 240, 240, 4])
    x = tf.reshape(x,[batch_size,240,240,-1])
    print_tensor_shape(x,'X despues del Reshaping')
    print("\n")
    #WHERE
    #SMALL
    conv1 = conv2d(x,weights['w_s1'],biases['b_s_1'],strides=1)
    print_tensor_shape(conv1,'Shape conv1')
    pool1 = maxpool2d(conv1,kernel_size=3,strides=1)
    print_tensor_shape(pool1,'Shape pool1')
    conv2 = conv2d(pool1,weights['w_s2'],biases['b_s_2'],strides=1)
    print_tensor_shape(conv2,'Shape conv2')
    pool2 = maxpool2d(conv2,kernel_size=3,strides=1)
    print_tensor_shape(pool2,'Shape pool2')
    conv3 = conv2d(pool2,weights['w_s3'],biases['b_s_3'],strides=1)
    print_tensor_shape(conv3,'Shape conv3')
    pool3 = maxpool2d(conv3,kernel_size=3,strides=1)
    print_tensor_shape(pool3,'Shape pool3')
    conv4 = conv2d(pool3,weights['w_s4'],biases['b_s_4'],strides=1)
    print_tensor_shape(conv4,'Shape conv4')
    pool4 = maxpool2d(conv4,kernel_size=3,strides=1)
    print_tensor_shape(pool4,'Shape pool4')
    conv5 = conv2d(pool4,weights['w_s5'],biases['b_s_5'],strides=1)
    print_tensor_shape(conv5,'Shape conv5')
    pool5 = maxpool2d(conv5,kernel_size=3,strides=1)
    print_tensor_shape(pool5,'Shape pool5')
    print("\n")
    #MEDIUM
    conv6 = conv2d(x,weights['w_m1'],biases['b_m_1'],strides=1)
    print_tensor_shape(conv6,'Shape conv6')
    pool6 = maxpool2d(conv6,kernel_size=3,strides=1)
    print_tensor_shape(pool6,'Shape pool6')
    conv7 = conv2d(pool6,weights['w_m2'],biases['b_m_2'],strides=1)
    print_tensor_shape(conv7,'Shape conv7')
    pool7 = maxpool2d(conv7,kernel_size=3,strides=1)
    print_tensor_shape(pool7,'Shape pool7')
    conv8 = conv2d(pool7,weights['w_m3'],biases['b_m_3'],strides=1)
    print_tensor_shape(conv8,'Shape conv8')
    pool8 = maxpool2d(conv8,kernel_size=3,strides=1)
    print_tensor_shape(pool8,'Shape pool8')
    conv9 = conv2d(pool8,weights['w_m4'],biases['b_m_4'],strides=1)
    print_tensor_shape(conv9,'Shape conv9')
    pool9 = maxpool2d(conv9,kernel_size=3,strides=1)
    print_tensor_shape(pool9,'Shape pool9')
    conv10 = conv2d(pool9,weights['w_m5'],biases['b_m_5'],strides=1)
    print_tensor_shape(conv10,'Shape conv10')
    pool10 = maxpool2d(conv10,kernel_size=3,strides=1)
    print_tensor_shape(pool10,'Shape pool10')
    print("\n")
    #LARGE
    conv11 = conv2d(x,weights['w_l1'],biases['b_l_1'],strides=1)
    print_tensor_shape(conv11,'Shape conv11')
    pool11 = maxpool2d(conv11,kernel_size=3,strides=1)
    print_tensor_shape(pool11,'Shape pool11')
    conv12 = conv2d(pool11,weights['w_l2'],biases['b_l_2'],strides=1)
    print_tensor_shape(conv12,'Shape conv12')
    pool12 = maxpool2d(conv12,kernel_size=3,strides=1)
    print_tensor_shape(pool12,'Shape pool12')
    conv13 = conv2d(pool12,weights['w_l3'],biases['b_l_3'],strides=1)
    print_tensor_shape(conv13,'Shape conv13')
    pool13 = maxpool2d(conv13,kernel_size=3,strides=1)
    print_tensor_shape(pool11,'Shape pool13')
    conv14 = conv2d(pool13,weights['w_l4'],biases['b_l_4'],strides=1)
    print_tensor_shape(conv12,'Shape conv14')
    pool14 = maxpool2d(conv14,kernel_size=3,strides=1)
    print_tensor_shape(pool14,'Shape pool14')
    conv15 = conv2d(pool14,weights['w_l5'],biases['b_l_5'],strides=1)
    print_tensor_shape(conv15,'Shape conv15')
    pool15 = maxpool2d(conv15,kernel_size=3,strides=1)
    print_tensor_shape(pool14,'Shape pool15')
    print("\n")
    #FUSION
    conv16 = tf.concat((pool5,pool10,pool15),3)
    print_tensor_shape(conv16,'Shape conv16_concat')
    conv16 = conv2d(conv16,weights['w_f1'],biases['b_f_1'],strides=1)
    print_tensor_shape(conv16,'Shape conv16')
    drop1 = tf.nn.dropout(conv16,dropout)
    print_tensor_shape(drop1,'Shape drop1')
    conv17 = conv2d(drop1,weights['w_f2'],biases['b_f_2'],strides=1)
    print_tensor_shape(conv17,'Shape conv17')
    drop2 = tf.nn.dropout(conv17,dropout)
    print_tensor_shape(drop2,'Shape drop2')
    conv18 = conv2d(drop2,weights['w_f3'],biases['b_f_3'],strides=1)
    print_tensor_shape(conv18,'Shape conv18')
    print("\n")
    #WHAT
    #LARGE
    conv1l =conv2d(x,weights['w_l6'],biases['b_l_6'],strides=1)
    print_tensor_shape(conv1l,'Shape conv1l')
    pool1l= maxpool2d(conv1l,kernel_size=2,strides=2)
    print_tensor_shape(pool1l,'Shape pool1l')
    conv2l =conv2d(pool1l,weights['w_l7'],biases['b_l_7'],strides=1)
    print_tensor_shape(conv2l,'Shape conv2l')
    pool2l= maxpool2d(conv2l,kernel_size=2,strides=2)
    print_tensor_shape(pool2l,'Shape pool2l')
    conv3l =conv2d(pool2l,weights['w_l8'],biases['b_l_8'],strides=1)
    print_tensor_shape(conv3l,'Shape conv3l')
    pool3l= maxpool2d(conv3l,kernel_size=2,strides=2)
    print_tensor_shape(pool3l,'Shape pool3l')
    conv4l =conv2d(pool3l,weights['w_l9'],biases['b_l_9'],strides=1)
    print_tensor_shape(conv4l,'Shape conv4l')
    print("\n")
    #MEDIUM
    conv1m =conv2d(x,weights['w_m6'],biases['b_m_6'],strides=1)
    print_tensor_shape(conv1m,'Shape conv1m')
    pool1m= maxpool2d(conv1m,kernel_size=2,strides=2)
    print_tensor_shape(pool1m,'Shape pool1m')
    conv2m = conv2d(pool1m,weights['w_m7'],biases['b_m_7'],strides=1)
    print_tensor_shape(conv2m,'Shape conv2m')
    pool2m= maxpool2d(conv2m,kernel_size=2,strides=2)
    print_tensor_shape(pool2m,'Shape pool2m')
    conv3m = conv2d(pool2m,weights['w_m8'],biases['b_m_8'],strides=1)
    print_tensor_shape(conv3m,'Shape conv3m')
    pool3m= maxpool2d(conv3m,kernel_size=2,strides=2)
    print_tensor_shape(pool3m,'Shape pool3m')
    conv4m = conv2d(pool3m,weights['w_m9'],biases['b_m_9'],strides=1)
    print_tensor_shape(conv4m,'Shape conv4m')
    print("\n")
    #SMALL
    conv1s =conv2d(x,weights['w_s6'],biases['b_s_6'],strides=1)
    print_tensor_shape(conv1s,'Shape conv1s')
    pool1s= maxpool2d(conv1s,kernel_size=2,strides=2)
    print_tensor_shape(pool1s,'Shape pool1s')
    conv2s = conv2d(pool1s,weights['w_s7'],biases['b_s_7'],strides=1)
    print_tensor_shape(conv2s,'Shape conv2s')
    pool2s= maxpool2d(conv2s,kernel_size=2,strides=2)
    print_tensor_shape(pool2s,'Shape pool2s')
    conv3s = conv2d(pool2s,weights['w_s8'],biases['b_s_8'],strides=1)
    print_tensor_shape(conv3s,'Shape conv3s')
    pool3s= maxpool2d(conv3s,kernel_size=2,strides=2)
    print_tensor_shape(pool3s,'Shape pool3s')
    conv4s = conv2d(pool3s,weights['w_s9'],biases['b_s_9'],strides=1)
    print_tensor_shape(conv4s,'Shape conv4s')
    print("\n")
    #FUSION
    convf1 = tf.concat((conv4l,conv4m,conv4s),3) 
    print_tensor_shape(convf1,'Shape convf1')
    convf2 = conv2d(convf1,weights['w_f4'],biases['b_f_4'],strides=1)
    print_tensor_shape(convf2,'Shape convf2')
    poolf2= maxpool2d(convf2,kernel_size=2,strides=2)
    print_tensor_shape(poolf2,'Shape poolf2')
    dropf3 = tf.nn.dropout(poolf2,dropout)
    print_tensor_shape(dropf3,'Shape dropf3')
    convf4 = conv2d(dropf3,weights['w_f5'],biases['b_f_5'],strides=1)
    print_tensor_shape(convf4,'Shape convf4')
    dropf5 = tf.nn.dropout(convf4,dropout)
    print_tensor_shape(dropf5,'Shape dropf5')
    convf5 = conv2d(dropf5,weights['w_f6'],biases['b_f_6'],strides=1)
    print_tensor_shape(convf5,'Shape convf5')
    print("\n")
    #DECONV
    deconv1= deconv2d(convf5,weights['w_fdec1'],output_shape = [batch_size,60,60,5],strides=4)
    print_tensor_shape(deconv1,'Shape deconv1')
    deconv2 = deconv2d(deconv1,weights['w_fdec2'],output_shape = [batch_size,240,240,5],strides=4)
    print_tensor_shape(deconv2,'Shape deconv2')
    print("\n")
    #FINAL FUSION
    conv_final_fusion1 = tf.concat((conv18,deconv2),3)
    print_tensor_shape(conv_final_fusion1,'Shape conv_final_fusion1')
    conv_final_fusion2 = conv2d(conv_final_fusion1,weights['w_final_fusion'],biases['b_ff'],strides=1)
    print_tensor_shape(conv_final_fusion2,'Shape conv_final_fusion2')
    print("\n")
    
    return conv_final_fusion2

In [10]:
learning_rate = 0.01
decay_rate = 1.0
decay_steps = 1000
num_epochs = 1
batch_size = 1
num_input = 57600
filename = 'C:\\Users\\rober\\Practicas\\training\\training_v2.tfrecords'
checkpoint = 'C:\\Users\\rober\\Practicas\\training\\checkpoint'
X = tf.placeholder(tf.float32, [None, num_input])

In [19]:
def run_training():
    images,labels = inputs(batch_size,num_epochs,filename)
    print_tensor_shape(images,'Images Despues de la funcion inputs')
    print_tensor_shape(labels,'Labels Despues de la funcion inputs')
    #results = inference(images)
    results = net(images,weights,biases,dropout)
    perdidas = loss(results,labels)
    train_op = training(perdidas,learning_rate,decay_steps,decay_rate)
    summary_op =tf.summary.merge_all()
    init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
    saver = tf.train.Saver()
    sess =tf.Session()
    summary_writer = tf.summary.FileWriter(checkpoint,sess.graph)
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    '''try:
        step = 0
        while not coord.should_stop():
            start_time = time.time()
            _,loss_value = sess.run([train_op,perdidas])
            duration = time.time()-start_time
            if step % 100 == 0:
                print('OUTPUT:Step %d: loss = %.3f (%.3f sec)' % (step,loss_value,duration))
                summary_str = sess.run(summary_op)
                summary_writer.add_summary(summary_str,step)
                summary_writer.flush()
            if step % 1000 == 0:
                checkpoint_path = os.path.join(checkpoint,'model.ckpt')
                saver.save(sess,checkpoint_path,global_step = step)
            step +=1
    except tf.errors.OutOfRangeError:
        print('OUTPUT: Done training for %d epochs, %d steps',num_epochs,step)
        checkpoint_path = os.path.join(checkpoint,'model.ckpt')
        saver.save(sess,checkpoint_path,global_step = step)
    #finally:'''
        coord.request_stop()
    coord.join(threads)
    sess.close()

In [20]:
def main(_):
    run_training()

In [21]:
if __name__ == '__main__':
    tf.app.run

In [22]:
main(_)

image (4, 240, 240) (1, 240, 240)
Images Despues de la funcion inputs (1, 4, 240, 240)
Labels Despues de la funcion inputs (1, 1, 240, 240)
X antes del Reshaping (1, 4, 240, 240)
X despues del Reshaping (1, 240, 240, 4)


Shape conv1 (1, 240, 240, 64)
Shape pool1 (1, 240, 240, 64)
Shape conv2 (1, 240, 240, 64)
Shape pool2 (1, 240, 240, 64)
Shape conv3 (1, 240, 240, 64)
Shape pool3 (1, 240, 240, 64)
Shape conv4 (1, 240, 240, 64)
Shape pool4 (1, 240, 240, 64)
Shape conv5 (1, 240, 240, 64)
Shape pool5 (1, 240, 240, 64)


Shape conv6 (1, 240, 240, 128)
Shape pool6 (1, 240, 240, 128)
Shape conv7 (1, 240, 240, 128)
Shape pool7 (1, 240, 240, 128)
Shape conv8 (1, 240, 240, 128)
Shape pool8 (1, 240, 240, 128)
Shape conv9 (1, 240, 240, 128)
Shape pool9 (1, 240, 240, 128)
Shape conv10 (1, 240, 240, 128)
Shape pool10 (1, 240, 240, 128)


Shape conv11 (1, 240, 240, 256)
Shape pool11 (1, 240, 240, 256)
Shape conv12 (1, 240, 240, 256)
Shape pool12 (1, 240, 240, 256)
Shape conv13 (1, 240, 240, 256)
Sh